In [1]:
import logging
import os
import sys
from datasets import load_metric, load_from_disk, Sequence, Value, Features, Dataset, DatasetDict

from transformers import AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer

from transformers import (
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)

sys.path.append('/opt/ml/code/')

from utils_qa import postprocess_qa_predictions, check_no_error, tokenize
from trainer_qa import QuestionAnsweringTrainer
from retrieval import SparseRetrieval

from arguments import (
    ModelArguments,
    DataTrainingArguments,
)

/usr/local/lib/mecab/dic/mecab-ko-dic
/opt/conda/lib/python3.7/site-packages/konlpy


In [3]:
parser = HfArgumentParser(
    (ModelArguments, DataTrainingArguments, TrainingArguments)
)

model_args, data_args, training_args = parser.parse_args_into_dataclasses(
    args=["--output_dir", "/opt/ml/code/outputs/notebook_test/", "--dataset_name",
         "/opt/ml/input/data/data/test_dataset/", "--model_name_or_path",
         "/opt/ml/code/models/train_dataset/checkpoint-1500/", "--do_predict"])

In [4]:
# 어따 쓰는 거지 => retriever에서 쓰인다.
training_args.do_train = True

In [5]:
set_seed(training_args.seed)

In [44]:
data_args.dataset_name

'/opt/ml/input/data/data/test_dataset/'

In [75]:
datasets = load_from_disk(data_args.dataset_name)

In [7]:
model_args.model_name_or_path

'/opt/ml/code/models/train_dataset/checkpoint-1500/'

In [8]:
config = AutoConfig.from_pretrained(
    model_args.config_name
    if model_args.config_name
    else model_args.model_name_or_path,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name
    if model_args.tokenizer_name
    else model_args.model_name_or_path,
    use_fast=True,
)
model = AutoModelForQuestionAnswering.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
)

In [9]:
data_args.eval_retrieval

True

```python
if data_args.eval_retrieval:
    datasets = run_sparse_retrieval(datasets, training_args)
```

#### run_sparse_retrieval

**SparseRetrieval**

In [10]:
retriever = SparseRetrieval(tokenize_fn=tokenize,
                            data_path="/opt/ml/input/data/data/",
                            context_path="wikipedia_documents.json")

Lengths of unique contexts : 56737


In [11]:
retriever.tfidfv

TfidfVectorizer(max_features=50000, ngram_range=(1, 2),
                tokenizer=<function tokenize at 0x7fa3f87c3290>)

In [14]:
retriever.p_embedding is None

True

In [16]:
retriever.indexer is None

True

**get_sparse_embedding**

In [17]:
retriever.get_sparse_embedding()

Embedding pickle load.


In [18]:
retriever.p_embedding

<56737x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 15905266 stored elements in Compressed Sparse Row format>

In [21]:
import numpy as np
retriever.p_embedding.astype(np.float32).todense()[0]

matrix([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [23]:
retriever.indexer is None

True

**retriever.retrieve**

(query: str)값이 들어오는 경우

In [25]:
retriever.get_relevant_doc("안녕하세요", k=1)

[transform] done in 0.003 s
[query ex search] done in 1.038 s


([0.2959788753283241], [24903])

In [26]:
retriever.get_relevant_doc("안녕하세요", k=3)

[transform] done in 0.003 s
[query ex search] done in 1.310 s


([0.2959788753283241, 0.20983814983342725, 0.2091296880230663],
 [24903, 11154, 43011])

In [30]:
doc_scores, doc_indices = retriever.get_relevant_doc("안녕하세요", k=3)

[transform] done in 0.002 s
[query ex search] done in 1.154 s


In [31]:
for i in range(3):
    print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
    print(retriever.contexts[doc_indices[i]])

Top-1 passage with score 0.2960
대한민국에서는 정식 발매되지 않았다.

* 안녕, 절망선생
:애니메이션 1기. 2007년 7월부터 9월까지의 방송을 4장에 나눠 발매했으며, 특장판과 일반판으로 나뉘여져있다.
* 절망소녀찬집
:애니메이션판 다이제스트 DVD. 애니메이션 1기보다 더욱 원작에 충실하며, 히토 나미의 이야기를 중간중간에 끼워넣었다. 2008년 1월 1일 발매.
* 속 안녕, 절망선생
:애니메이션 2기. 2008년 1월부터 3월까지의 방송을 4장에 나눠 발매했으며, 특장판과 일반판으로 나뉘여져있다.
* 속 절망소녀찬집
:애니메이션판 다이제스트 DVD. 절망소녀찬집의 후속작이며, 애니메이션 2기의 내용보다 더욱더 원작에 충실한 내용들로 구성되어 있다. 2008년 8월 27일 발매. 특전영상으로 절망문학집, 에피소드 『은폐졸』의 오리지널 성우진 버전을 수록하였다.(원작에는 성우진 캐스팅을 바꿔서 녹음하였다.)
* 옥 안녕, 절망선생
:안녕, 절망선생의 오리지널 애니메이션 DVD. 3장으로 구성되어있으며, 각각 2008년 10월 17일, 2008년 12월 10일, 2009년 2월 17일에 발매되었다.
* 참 안녕, 절망선생
:애니메이션 3기. 2009년 7월부터 9월까지의 방송을 4장에 나눠 발매했으며, 특장판과 일반판으로 나뉘여져있다.
* 참 안녕, 절망선생 ~ 번외편
:안녕, 절망선생의 오리지널 애니메이션 DVD. 상(上),하(下)편으로 구성되어있다. 각각 2009년 11월 17일, 2010년 2월 17일날 발매되었다.
* 안녕, 절망선생 Blu - ray BOX
:안녕, 절망선생의 애니메이션 1기의 블루레이판. 디스크 2매로 구성되어있으며 기간한정품이다. 2011년 3월 23일에 발매되었다.
* 속 안녕, 절망선생 Blu - ray BOX
:안녕, 절망선생의 애니메이션 2기의 블루레이판. 디스크 3매로 구성되어있으며 기간한정품이다. 2011년 5월 25일에 발매되었다.
* 참 안녕, 절망선생 Blu - ray BOX
:안녕, 

In [ ]:
# return doc_socres, contexts

(dataset: Dataset)이 들어 오는 경우

In [38]:
import time
from contextlib import contextmanager

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.3f} s')

In [36]:
class custom_timer:
    def __enter__(self):
        pass
    def __exit__(self):
        pass

In [43]:
datasets['validation']

Dataset({
    features: ['id', 'question'],
    num_rows: 600
})

In [45]:
total = []

with timer("query exhaustive search"):
    doc_scores, doc_indices = retriever.get_relevant_doc_bulk(
        datasets['validation']['question'], k=1)

[query exhaustive search] done in 7.988 s


In [48]:
# 600개 문서의 top_k개 만큼의 관련 문서를 가져온다.
print(len(doc_scores), doc_indices[:3])

600 [[23174], [23174], [33861]]


**doc_score, doc_index 변환 one_step**

In [52]:
idx, example = 0, next(iter(datasets['validation']))

In [53]:
example

{'question': "유령'은 어느 행성에서 지구로 왔는가?", 'id': 'mrc-1-000653'}

In [54]:
tmp = {
    "question": example["question"],
    "id": example['id'],
    "context_id": doc_indices[idx][0],  # retrieved id
    "context": retriever.contexts[doc_indices[idx][0]]  # retrieved doument
}

In [58]:
tmp

{'question': "유령'은 어느 행성에서 지구로 왔는가?",
 'id': 'mrc-1-000653',
 'context_id': 23174,
 'context': '베피콜롬보\n베피콜롬보는 수성 탐사 계획 중 하나로 ESA와 JAXA가 공동으로 계획했다. 소형 탐사선 2기를 보유하고 있으며, 유럽(MPO)과 일본MMO)에서 각각 한 기씩 제공했으며, 또한 한 기는 사진을 찍고, 다른 한 기는 자기장을 연구하는 등 역할이 확실히 구별되어 있다. \n\n#태양 성운, 행성계에 있어서, 수성에 대해 연구해야 할 것은 무엇인가?\n#왜 수성의 밀도는 다른 지구형 행성보다 높은가?\n#수성의 핵은 액체인가? 고체인가?\n#오늘날도 수성 구조는 활동적인가?\n#금성과 화성, 달도 가지고 있지 못 한 작은 행성이 왜 자기장을 가지고 있는가?\n#수성의 주 성분이 철임에도, 분광 관측으로는 발견되지 않았던 이유는 무엇인가?\n#극점의 영구 동토에는 황 혹은 얼음이 존재하는가?\n#외기권의 형성 원리는 무엇인가?\n#이온층이 없는데도, 자기장과 태양풍이 어떻게 상호 작용을 하는가?\n#수성의 자화(磁化)된 환경이 지구에서 관측되는 오로라, 밴 앨랜대, 자기 폭풍 등이 존재한다는 것을 암시하는가?\n#공간의 왜곡으로 인한 수성의 근일점 변화가 일반상대성이론에 근거한 결과의 오차값을 더 줄일 수 있는가\n\n매리너 10호나 메신저와 같이, 베피콜롬보는 금성과 지구에서 플라이바이를 사용할 예정이다. 특히, 태양 에너지 추진을 이용하여 달, 금성을 지나 수성에 느린 속도로 도달 할 전망이다. 이런 기술은 태양 중력의 영향을 최소화하여 수성에 접근하기 위해서는 필수적이다\n\n베피콜롬보는 2018년 10월 경에 발사 되어, 2025년 12월 5일, 수성 궤도로 진입 할 예정이다. 그 후, 2년동안 수성에 대한 정보를 모으고 연구를 행할 것이다.'}

In [59]:
example.keys()

dict_keys(['question', 'id'])

In [60]:
'context' in example.keys() and 'answers' in example.keys()

False

```python
if 'context' in example.keys() and 'answers' in example.keys():
    tmp["original_context"] = example['context']  # original document
    tmp["answers"] = example['answers']           # original answer
```

(evaluation시에 쓰임) answers는 점수 계산에 쓰이고 original_context는 저장은 하나 사용하는 경우는 없다.

In [61]:
total = []
for idx, example in enumerate(datasets['validation']):
    tmp = {
        "question": example["question"],
        "id": example['id'],
        "context_id": doc_indices[idx][0],  # retrieved id
        "context": retriever.contexts[doc_indices[idx][0]]  # retrieved doument
    }
    if 'context' in example.keys() and 'answers' in example.keys():
        tmp["original_context"] = example['context']  # original document
        tmp["answers"] = example['answers']           # original answer
    total.append(tmp)

In [64]:
import pandas as pd
cqas = pd.DataFrame(total)

```python
df = retriever.retrieve(datasets['validation'])
# df = retriever.retrieve_faiss(dataset['validation'])
```

In [65]:
df = cqas
df.head(3)

,question,id,context_id,context
0,유령'은 어느 행성에서 지구로 왔는가?,mrc-1-000653,23174,베피콜롬보\n베피콜롬보는 수성 탐사 계획 중 하나로 ESA와 JAXA가 공동으로 계...
1,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,mrc-1-001113,23174,베피콜롬보\n베피콜롬보는 수성 탐사 계획 중 하나로 ESA와 JAXA가 공동으로 계...
2,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,mrc-0-002191,33861,피쉬의 후기작들 다수는 스포큰 워드를 가사에 포함한다. 초기 마릴리온의 곡에서도 그...


```python
    if training_args.do_predict: # test data 에 대해선 정답이 없으므로 id question context 로만 데이터셋이 구성됩니다.
f = Features({'context': Value(dtype='string', id=None),
              'id': Value(dtype='string', id=None),
              'question': Value(dtype='string', id=None)})

elif training_args.do_eval: # train data 에 대해선 정답이 존재하므로 id question context answer 로 데이터셋이 구성됩니다.
f = Features({'answers': Sequence(feature={'text': Value(dtype='string', id=None),
                                           'answer_start': Value(dtype='int32', id=None)},
                                  length=-1, id=None),
              'context': Value(dtype='string', id=None),
              'id': Value(dtype='string', id=None),
              'question': Value(dtype='string', id=None)})
```

In [68]:
f = Features({'context': Value(dtype='string', id=None),
              'id': Value(dtype='string', id=None),
              'question': Value(dtype='string', id=None)})

In [71]:
f

{'context': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None)}

In [76]:
datasets

DatasetDict({
    validation: Dataset({
        features: ['id', 'question'],
        num_rows: 600
    })
})

In [77]:
new_datasets = DatasetDict({'validation': Dataset.from_pandas(df, features=f)})

In [78]:
new_datasets

DatasetDict({
    validation: Dataset({
        features: ['context', 'id', 'question'],
        num_rows: 600
    })
})

### run_mrc

In [79]:
column_names = new_datasets["validation"].column_names

question_column_name = "question" if "question" in column_names else column_names[0]
context_column_name = "context" if "context" in column_names else column_names[1]
answer_column_name = "answers" if "answers" in column_names else column_names[2]

# Padding side determines if we do (question|context) or (context|question).
pad_on_right = tokenizer.padding_side == "right"

# check if there is an error
last_checkpoint, max_seq_length = check_no_error(training_args, data_args, tokenizer, datasets)

In [83]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples[question_column_name if pad_on_right else context_column_name],
        examples[context_column_name if pad_on_right else question_column_name],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_seq_length,
        stride=data_args.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length" if data_args.pad_to_max_length else False,
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]
    return tokenized_examples

eval_dataset = new_datasets["validation"]

# Validation Feature Creation
eval_dataset = eval_dataset.map(
    prepare_validation_features,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
    remove_columns=column_names,
    load_from_cache_file=not data_args.overwrite_cache,
)

# Data collator
# We have already padded to max length if the corresponding flag is True, otherwise we need to pad in the data collator.
data_collator = (
    DataCollatorWithPadding(
        tokenizer, pad_to_multiple_of=8 if training_args.fp16 else None
    )
)

# Post-processing:
def post_processing_function(examples, features, predictions, training_args):
    # Post-processing: we match the start logits and end logits to answers in the original context.
    predictions = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        max_answer_length=data_args.max_answer_length,
        output_dir=training_args.output_dir,
    )
    # Format the result to the format the metric expects.
    formatted_predictions = [
        {"id": k, "prediction_text": v} for k, v in predictions.items()
    ]
    if training_args.do_predict:
        return formatted_predictions

    elif training_args.do_eval:
        references = [
            {"id": ex["id"], "answers": ex[answer_column_name]}
            for ex in datasets["validation"]
        ]
        return EvalPrediction(predictions=formatted_predictions, label_ids=references)

metric = load_metric("squad")

def compute_metrics(p: EvalPrediction):
    return metric.compute(predictions=p.predictions, references=p.label_ids)

print("init trainer...")
# Initialize our Trainer
trainer = QuestionAnsweringTrainer(
    model=model,
    args=training_args,
    train_dataset= None,
    eval_dataset=eval_dataset,
    eval_examples=datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics,
)

#### eval dataset & eval example - will create predictions.json
if training_args.do_predict:
    predictions = trainer.predict(test_dataset=eval_dataset,
                                    test_examples=new_datasets['validation'])

    # predictions.json is already saved when we call postprocess_qa_predictions(). so there is no need to further use predictions.
    print("No metric can be presented because there is no correct answer given. Job done!")



init trainer...



No metric can be presented because there is no correct answer given. Job done!


In [84]:
predictions

[{'id': 'mrc-1-000653', 'prediction_text': '유럽(MPO)과 일본MMO)'},
 {'id': 'mrc-1-001113', 'prediction_text': '유럽(MPO)과 일본MMO)'},
 {'id': 'mrc-0-002191', 'prediction_text': '제네시스 (밴드)'},
 {'id': 'mrc-0-003951', 'prediction_text': '“미슈텍어”'},
 {'id': 'mrc-1-001272', 'prediction_text': '대통령에 의하여'},
 {'id': 'mrc-1-000993', 'prediction_text': "'trema'"},
 {'id': 'mrc-0-005021', 'prediction_text': '『사회주의적 민주주의에 대하여』'},
 {'id': 'mrc-1-000163', 'prediction_text': '유럽(MPO)과 일본MMO)'},
 {'id': 'mrc-0-001283', 'prediction_text': '후에 임오군란(1882)'},
 {'id': 'mrc-0-004543', 'prediction_text': '제3궤조'},
 {'id': 'mrc-0-000439', 'prediction_text': '나중에'},
 {'id': 'mrc-0-002895', 'prediction_text': '고슴도치'},
 {'id': 'mrc-0-000535', 'prediction_text': '"옵션 아웃"'},
 {'id': 'mrc-1-001724', 'prediction_text': "'역사가는 이따금 소문에 의한 증거를 사용할 수 있음'"},
 {'id': 'mrc-0-000901', 'prediction_text': '유럽(MPO)과 일본MMO)'},
 {'id': 'mrc-0-001606', 'prediction_text': "'로마네스크'란 용어 자체가 '로마적'"},
 {'id': 'mrc-0-000266', 'prediction_text': 